In [1]:
import pandas as pd
import numpy as np
add_url='C:/Users/SAMSUNG/Desktop/DataAnalysis/my_projects/02_semi_project2/'
Data=pd.read_excel("C:/Users/SAMSUNG/Desktop/DataAnalysis/my_projects/02_semi_project2/관광지.xlsx")
data=pd.DataFrame(Data)

> 겹치는게 없다..

### 데이터 전처리
>제주도 주소가 아닌 곳을 제외

In [2]:
data=data[data["지번주소"].str.find("제주특별")!=-1]
data.drop(columns="Unnamed: 0",inplace=True)
data["행정구역"]=np.nan

> 진짜 고생했네 ㅜㅜㅋㅋ

In [3]:
행정구역 = {'구좌':'구좌',
  '조천':'조천', 
  '화북': '동제주','봉개':'동제주', '회천':'동제주','용강':'동제주',
  '오라':'중제주','아라':'중제주','오등':'중제주','화북':'중제주','이도':'중제주','건입':'중제주','삼도':'중제주','용담':'중제주','일도':'중제주', 
  '도두':'서제주','이호':'서제주','외도':'서제주','내도':'서제주','도평':'서제주','노형':'서제주','연동':'서제주','강정':'서제주','도순':'서제주',
  '영남':'서제주','월평':'서제주','해안':'서제주', '노형':'서제주',
  '애월' :'애월',
  '성산':'성산',
  '한림':'한림',
  '표선':'표선',
  '남원':'남원',
  '한경':'한경',
  '안덕':'안덕',
  '영천':'동서귀','동흥':'동서귀','효돈':'동서귀','송산':'동서귀','신효':'동서귀','하효':'동서귀','서귀':'동서귀','보목':'동서귀','동홍':'동서귀',
  '대천':'중서귀','대륜':'중서귀','서홍':'중서귀', '강정':'중서귀','법환':'중서귀','서호':'중서귀','호근':'중서귀',
  '여래':'서서귀','중문':'서서귀','회수':'서서귀', '하원':'서서귀','대포':'서서귀', '색달':'서서귀', '하예':'서서귀', '상예':'서서귀',  
  '대정':'대정'}

In [4]:
def trans(x):
    for i in 행정구역.keys():
        try:
            if i in x:
                return 행정구역[i]
        except:
            break

In [5]:
# 지번주소
error=[]
data["행정구역"]=data["지번주소"].apply(lambda x:trans(x))

In [6]:
# 도로명주소
data.loc[data["행정구역"].isna(),"행정구역"]=data[data["행정구역"].isna()].apply(lambda x:trans(x))

### 시각화 작업

In [7]:
from IPython.display import SVG
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import os

#### 행정구역별

In [8]:
colors = ['#ffeecc', '#ffd480', '#ffbb33', '#ffaa00', '#e69900']

#### histogram 을 이용한 구간 설정
> 좀더 상세히 혹은 덜 나눠야 보기 좋을듯 

In [9]:
color_rule=pd.merge(pd.DataFrame(set(행정구역.values())).set_index(0),pd.DataFrame(data["행정구역"].value_counts()),
                    how="left",left_index=True,right_index=True).fillna(0)

In [10]:
x,y=np.histogram(color_rule["count"].values,bins=5)

In [11]:
y

array([  0. ,  50.8, 101.6, 152.4, 203.2, 254. ])

In [12]:
dc=dict(zip(y[:-1],colors))

In [13]:
for name in color_rule.index:
    for i in y[:-1]:
        if color_rule.loc[name]['count']>=i:
            color_rule.loc[name,'color_id']=i

색상 입히기

In [14]:
color_rule

,count,color_id
0,,
안덕,104.0,101.6
한경,60.0,50.8
동제주,22.0,0.0
조천,98.0,50.8
중제주,108.0,101.6
애월,108.0,101.6
서서귀,0.0,0.0
남원,63.0,50.8
대정,0.0,0.0


In [15]:
with open("./제주.svg","r",encoding="utf-8") as f:
    jeju_map=f.readlines()

In [16]:
for name in color_rule.index:
    idx=[]
    for i in range(len(jeju_map)):
        if name in jeju_map[i]:
            idx.append(i)
    for j in map(lambda x:x+1,idx):
        jeju_map[j]=jeju_map[j].replace("#eeeeee",dc[color_rule.loc[name]["color_id"]])

In [17]:
with open("관광지시각화test.html","w") as f:
    for line in jeju_map:
        f.write(line)

In [18]:
set(행정구역.values())

{'구좌',
 '남원',
 '대정',
 '동서귀',
 '동제주',
 '서서귀',
 '서제주',
 '성산',
 '안덕',
 '애월',
 '조천',
 '중서귀',
 '중제주',
 '표선',
 '한경',
 '한림'}

In [19]:
data["행정구역"].value_counts()

행정구역
동서귀    254
애월     108
중제주    108
안덕     104
조천      98
서제주     95
구좌      95
한림      68
성산      67
남원      63
한경      60
표선      55
동제주     22
중서귀     11
Name: count, dtype: int64